---
<font size="20"><b>.BMP to Triangles :)</b></font>
A Python 3.9 Notebook by JPR

---

%60 of code is mangled together from SO

In [215]:
def read_rows(path):
    image_file = open(path, "rb")
    image_file.seek(10)
    addy = int.from_bytes(image_file.read(4), byteorder='little')
    image_file.seek(18)
    wid = int.from_bytes(image_file.read(4), byteorder='little')
    image_file.seek(22)
    hig = int.from_bytes(image_file.read(4), byteorder='little')
    print(f"w:{wid}; h:{hig}")
    image_file.seek(addy)

    # We need to read pixels in as rows to later swap the order
    # since BMP stores pixels starting at the bottom left.
    rows = []
    row = []
    pixel_index = 0

    while True:
        if pixel_index == wid:
            image_file.seek((4-(wid*3)%4), 1)
#             print("\n")
            pixel_index = 0
            rows.insert(0, row)
            if len(row) != wid:
                raise Exception(f"Row length is not {wid}*3 but " + str(len(row)) + " / 3.0 = " + str(len(row) / 3.0))
            row = []
        if len(rows) >= hig:
          break
        pixel_index += 1

        r_string = image_file.read(1)
        g_string = image_file.read(1)
        b_string = image_file.read(1)
        
        if len(r_string) == 0:
            # This is expected to happen when we've read everything.
            if len(rows) != hig:
                print("Warning!!! Read to the end of the file at the correct sub-pixel (red) but we've not read 1080 rows!")
            break

        if len(g_string) == 0:
            print("Warning!!! Got 0 length string for green. Breaking.")
            break

        if len(b_string) == 0:
            print("Warning!!! Got 0 length string for blue. Breaking.")
            break

        r = ord(r_string)
        g = ord(g_string)
        b = ord(b_string)
#         print(f"({r},{g},{b})", end=" ")
        row.append((b,g,r))
    image_file.close()

    return rows

In [216]:
from io import BytesIO
import math
import cairo
import IPython.display

WIDTH, HEIGHT = 256, 256

def parrow(num):
  if num == 1 or num == 2:
    return 1
  return 0

def normcol(col):
  return (col[0]/255, col[1]/255, col[2]/255)

svgio = BytesIO()
with cairo.SVGSurface(svgio, 800,800) as surface:
    # These lines are copied verbatim from the
    # pycairo page: https://pycairo.readthedocs.io
    context = cairo.Context(surface)
    context.scale(800, 800)
    context.set_line_width(0.01)
    def drawtriangle(x,y,s,par,col):
      c = 0.05*s
      context.move_to(x-0.866*c, y-0.5*c*par)
      context.line_to(x+s+0.866*c, y-0.5*c*par)
      context.line_to(x+s/2, y+3**(1/2)*s/2*par+c)
      context.close_path()
      context.set_source_rgb(*col)
      context.fill()
    
    def drawtgrid(top, bot, lmargin, rows):
      print(len(rows))
      rowlen = len(rows[0])
      h = (bot - top) * 2 / len(rows)
      s = h * 2 / 3**(1/2)
      y = top
      for i,row in enumerate(rows):
        x = lmargin + parrow(i%4)*s/2
        tripar = (i+1)%2 * 2 - 1
        for cell in row:
          drawtriangle(x,y,s,tripar,normcol(cell))
          x += s
        y += h*((i+1)%2)
      
    drawtgrid(0.1, 0.9, 0.1, read_rows("scram.bmp"))
    
IPython.display.SVG(data=svgio.getvalue())

w:77; h:200
200
